In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import OrderedDict

In [23]:
# shape (N, Cin, T) where Cin ~ dimensions of data (activity counts and light intensity)
X = torch.rand((32, 2, 10080))

# basic LSTM

In [24]:
X.transpose(2, 1).shape

torch.Size([32, 10080, 2])

In [25]:
device = torch.device('cuda:0')

In [26]:
d_in = 2
d_hidden = 128
nb_layers = 2
T = 10080
d_out = 1

class LSTM(nn.Module):
    '''Basic comparison for a 2 layer, final hidden unit clf using LSTM
    '''
    def __init__(self, d_in, d_hidden, nb_layers, T, d_out, regression=True):
        super().__init__()
        self.d_in = d_in
        self.d_hidden = d_hidden
        self.nb_layers = nb_layers
        self.T = T
        self.d_out = d_out
        
        # blocks
        self.lstm = nn.LSTM(d_in, d_hidden, num_layers=nb_layers, batch_first=True)
        self.pred = nn.Sequential(
            nn.Linear(d_hidden, d_hidden),
            nn.LeakyReLU(),
            nn.Linear(d_hidden, d_hidden//2),
            nn.LeakyReLU(),
            nn.Linear(d_hidden//2, d_out)
        )
    
    def forward(self, X):
        # X shape (N, d_in, T) where d_in=data dim (activity counts and light intensity)
        _, (h_n, _) = self.lstm(X.transpose(2, 1))
        return self.pred(h_n[-1, :, :])


In [27]:
net = LSTM(2, 128, 3, 10080, 3).to(device)

In [28]:
net(X.to(device)).shape

torch.Size([32, 3])

# basic CNN

In [29]:
class CNN(nn.Module):
    '''Call it VGG-1D'''
    def __init__(self, in_channels, L, d_out, conv_arch):
        '''
        Arguments:
          L (int): length of input sequence, needed to determine how to flatten VGG output
          conv_arch (list of tuples): e.g., [(1, 64), (1, 128)] where the first value is 
            the nb_convs and the second value is the d_out
        '''
        super().__init__()
        self.d_out = d_out
        conv_blocks = []
        nb_halves = 0
        for (num_convs, out_channels) in conv_arch:
            nb_halves = nb_halves + num_convs
            conv_blocks.append(self.vgg_block(num_convs, in_channels, out_channels))
            in_channels = out_channels
        self.vgg = nn.Sequential(*conv_blocks, nn.Flatten())
        self.pred = nn.Sequential(
            nn.Linear(out_channels*(L // (2**nb_halves)), 512),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Linear(512, d_out),
            )
        
    def vgg_block(self, nb_convs, in_channels, out_channels):
        layers = []
        for _ in range(nb_convs):
            layers.append(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1) 
                )
            layers.append(nn.LeakyReLU()) # add dropout?
            in_channels = out_channels
            layers.append(nn.MaxPool1d(kernel_size=2, stride=2))
        return nn.Sequential(*layers)
        
        
    def forward(self, X):
        # X.shape = (N, C, T)
        X = self.vgg(X)
        return self.pred(X)
        

In [30]:
net = CNN(2, 10080, 3, [(1, 32), (1, 64), (1, 128), (3, 256)])
net(X)

/home/ngrav/miniconda3/envs/wearables/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[-0.0412,  0.0086, -0.0419],
        [-0.0422,  0.0140, -0.0393],
        [-0.0349,  0.0071, -0.0353],
        [-0.0408,  0.0112, -0.0340],
        [-0.0419,  0.0183, -0.0394],
        [-0.0405,  0.0152, -0.0379],
        [-0.0412,  0.0053, -0.0407],
        [-0.0374,  0.0158, -0.0388],
        [-0.0432,  0.0071, -0.0427],
        [-0.0431,  0.0110, -0.0387],
        [-0.0385,  0.0122, -0.0393],
        [-0.0433,  0.0075, -0.0408],
        [-0.0490,  0.0221, -0.0331],
        [-0.0453,  0.0112, -0.0365],
        [-0.0401,  0.0075, -0.0403],
        [-0.0456,  0.0102, -0.0414],
        [-0.0436,  0.0081, -0.0401],
        [-0.0360,  0.0148, -0.0388],
        [-0.0424,  0.0138, -0.0370],
        [-0.0445,  0.0209, -0.0369],
        [-0.0414,  0.0056, -0.0341],
        [-0.0378,  0.0084, -0.0303],
        [-0.0382,  0.0027, -0.0417],
        [-0.0389,  0.0147, -0.0286],
        [-0.0456,  0.0103, -0.0370],
        [-0.0418,  0.0132, -0.0359],
        [-0.0431,  0.0054, -0.0352],
 

# DeepSleepNet

[Paper](https://arxiv.org/abs/1703.04046) using CNNs to reduce dimensionality, RNNs for sequential learning, and a residual connection for good measure



In [163]:
hn.shape

torch.Size([4, 32, 128])

In [31]:
t = torch.rand(32, 256, 103)

In [42]:
_, (h_n, _) = nn.LSTM(256, 256, num_layers=3, batch_first=True, bidirectional=True)(t.transpose(2, 1))

In [43]:
h_n.shape

torch.Size([6, 32, 256])

In [41]:
h_n.view(2, 2, -1, 256)[-1, :, :, :].reshape(-1, 512)

tensor([[-0.0253, -0.0010, -0.0019,  ...,  0.0256,  0.0109, -0.0174],
        [-0.0149, -0.0392,  0.0701,  ...,  0.0326,  0.0034, -0.0393],
        [-0.0107, -0.0310,  0.0404,  ...,  0.0503, -0.0245, -0.0517],
        ...,
        [-0.0643,  0.0548, -0.0918,  ..., -0.0344,  0.1019, -0.1268],
        [-0.0604,  0.0172, -0.0869,  ..., -0.0224,  0.1170, -0.1193],
        [-0.0571,  0.0537, -0.0761,  ..., -0.0286,  0.1008, -0.1110]],
       grad_fn=<ViewBackward>)

In [51]:
class DeepSleepNet(nn.Module):
    def __init__(self, d_in, L, d_out, regression=True):
        '''
        Arguments:
          L (int): length of input sequence
        '''
        super().__init__()
        
        # represention 
        self.lofi = nn.Sequential(
            self.conv_blk(d_in, 64, 60, 5, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 64, 60, 5, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 128, 2, 1, 1),
            self.conv_blk(128, 128, 2, 1, 1),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.hifi = nn.Sequential(
            self.conv_blk(d_in, 64, 5, 1, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 64, 5, 1, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 64, 5, 1, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 64, 5, 1, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 64, 5, 1, 0), 
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Dropout(),
            self.conv_blk(64, 128, 2, 1, 1),
            self.conv_blk(128, 128, 2, 1, 1),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # sequential learning
        self.lofi_rnn = nn.LSTM(128, 128, num_layers=2, batch_first=True, bidirectional=True, dropout=0.5)
#         self.lofi_fc1(128, 256) # how would you do this? ave or for loop over the seq?
        self.hifi_rnn = nn.LSTM(128, 128, num_layers=2, batch_first=True, bidirectional=True, dropout=0.5)
#         self.hifi_fc1 = nn.Linear(128, 256)

        # pred
        self.pred = nn.Sequential(
            nn.Linear(512, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, d_out)
        )
        
        
            
    def conv_blk(self, in_channels, nb_filters, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv1d(in_channels, nb_filters, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm1d(nb_filters), 
            nn.LeakyReLU())
        
    def forward(self, X):
        lo = self.lofi(X)
        hi = self.hifi(X)
        _, (lo, _) = self.lofi_rnn(lo.transpose(2, 1))
        lo = lo.view(2, 2, -1, 128)[-1, :, :, :].reshape(-1, 256) # (nb_layers, directions, N, L)
        _, (hi, _) = self.hifi_rnn(hi.transpose(2, 1))
        hi = hi.view(2, 2, -1, 128)[-1, :, :, :].reshape(-1, 256) # (nb_layers, directions, N, L)
        return self.pred(torch.cat((lo, hi), dim=-1))
#         X = torch.cat((self.lofi(X), self.hifi(X)), dim=-1)
#         X = F.dropout(X)
        
        
#         print(X.shape)
        # X shape (N, C, T)

In [52]:
net = DeepSleepNet(2, 64, 3)
net(X).shape

torch.Size([32, 3])

# ConvTransformer

In [55]:
t.shape

torch.Size([32, 256, 103])

In [59]:
out = nn.Transformer(d_model=256, dim_feedforward=512, batch_first=True)(t.transpose(2, 1), t.transpose(2, 1))
out.shape

torch.Size([32, 103, 256])

# ConvLSTM/GRU 

## (actual)
Top [reference](https://github.com/jhhuang96/ConvLSTM-PyTorch/blob/master/utils.py) GitHub result from Google

Call seems to be: 
```
encoder = Encoder(encoder_params[0], encoder_params[1]).cuda()
decoder = Decoder(decoder_params[0], decoder_params[1]).cuda()
net = ED(encoder, decoder)
```

See [`main.py`](https://github.com/jhhuang96/ConvLSTM-PyTorch/blob/master/main.py) for params

In [19]:
def make_layers(block):
    layers = []
    for layer_name, v in block.items():
        if 'pool' in layer_name:
            layer = nn.MaxPool2d(kernel_size=v[0], stride=v[1], padding=v[2])
            layers.append((layer_name, layer))
        elif 'deconv' in layer_name:
            transposeConv2d = nn.ConvTranspose2d(in_channels=v[0],
                                                 out_channels=v[1],
                                                 kernel_size=v[2],
                                                 stride=v[3],
                                                 padding=v[4])
            layers.append((layer_name, transposeConv2d))
            if 'relu' in layer_name:
                layers.append(('relu_' + layer_name, nn.ReLU(inplace=True)))
            elif 'leaky' in layer_name:
                layers.append(('leaky_' + layer_name,
                               nn.LeakyReLU(negative_slope=0.2, inplace=True)))
        elif 'conv' in layer_name:
            conv2d = nn.Conv2d(in_channels=v[0],
                               out_channels=v[1],
                               kernel_size=v[2],
                               stride=v[3],
                               padding=v[4])
            layers.append((layer_name, conv2d))
            if 'relu' in layer_name:
                layers.append(('relu_' + layer_name, nn.ReLU(inplace=True)))
            elif 'leaky' in layer_name:
                layers.append(('leaky_' + layer_name,
                               nn.LeakyReLU(negative_slope=0.2, inplace=True)))
        else:
            raise NotImplementedError
    return nn.Sequential(OrderedDict(layers))

class Encoder(nn.Module):
    def __init__(self, subnets, rnns):
        super().__init__()
        assert len(subnets) == len(rnns)
        self.blocks = len(subnets)

        for index, (params, rnn) in enumerate(zip(subnets, rnns), 1):
            # index sign from 1
            setattr(self, 'stage' + str(index), make_layers(params))
            setattr(self, 'rnn' + str(index), rnn)

    def forward_by_stage(self, inputs, subnet, rnn):
        seq_number, batch_size, input_channel, height, width = inputs.size()
        inputs = torch.reshape(inputs, (-1, input_channel, height, width))
        inputs = subnet(inputs)
        inputs = torch.reshape(inputs, (seq_number, batch_size, inputs.size(1),
                                        inputs.size(2), inputs.size(3)))
        outputs_stage, state_stage = rnn(inputs, None)
        return outputs_stage, state_stage

    def forward(self, inputs):
        inputs = inputs.transpose(0, 1)  # to S,B,1,64,64
        hidden_states = []
        logging.debug(inputs.size())
        for i in range(1, self.blocks + 1):
            inputs, state_stage = self.forward_by_stage(
                inputs, getattr(self, 'stage' + str(i)),
                getattr(self, 'rnn' + str(i)))
            hidden_states.append(state_stage)
        return tuple(hidden_states)
    
class Decoder(nn.Module):
    def __init__(self, subnets, rnns):
        super().__init__()
        assert len(subnets) == len(rnns)

        self.blocks = len(subnets)

        for index, (params, rnn) in enumerate(zip(subnets, rnns)):
            setattr(self, 'rnn' + str(self.blocks - index), rnn)
            setattr(self, 'stage' + str(self.blocks - index),
                    make_layers(params))

    def forward_by_stage(self, inputs, state, subnet, rnn):
        inputs, state_stage = rnn(inputs, state, seq_len=10)
        seq_number, batch_size, input_channel, height, width = inputs.size()
        inputs = torch.reshape(inputs, (-1, input_channel, height, width))
        inputs = subnet(inputs)
        inputs = torch.reshape(inputs, (seq_number, batch_size, inputs.size(1),
                                        inputs.size(2), inputs.size(3)))
        return inputs

        # input: 5D S*B*C*H*W

    def forward(self, hidden_states):
        inputs = self.forward_by_stage(None, hidden_states[-1],
                                       getattr(self, 'stage3'),
                                       getattr(self, 'rnn3'))
        for i in list(range(1, self.blocks))[::-1]:
            inputs = self.forward_by_stage(inputs, hidden_states[i - 1],
                                           getattr(self, 'stage' + str(i)),
                                           getattr(self, 'rnn' + str(i)))
        inputs = inputs.transpose(0, 1)  # to B,S,1,64,64
        return inputs


class activation():

    def __init__(self, act_type, negative_slope=0.2, inplace=True):
        super().__init__()
        self._act_type = act_type
        self.negative_slope = negative_slope
        self.inplace = inplace

    def __call__(self, input):
        if self._act_type == 'leaky':
            return F.leaky_relu(input, negative_slope=self.negative_slope, inplace=self.inplace)
        elif self._act_type == 'relu':
            return F.relu(input, inplace=self.inplace)
        elif self._act_type == 'sigmoid':
            return torch.sigmoid(input)
        else:
            raise NotImplementedError


class ED(nn.Module):

    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input):
        state = self.encoder(input)
        output = self.decoder(state)
        return output

torch.Size([2, 32, 16])

In [ ]:
# Encoder/Decoder with CLSTM_cell?

In [5]:
X_transform = X.reshape(-1, 2, 7, 24, 60).shape

torch.Size([32, 2, 7, 24, 60])